In [ ]:
#| default_exp vanity

# Vanity addresses

> Use the notifyr bot to generate vanity addresses and get notified by DM when they are done.

## Run a simple generator that finds vanity addresses
Here is [an interesting article by Kris Constable about vanity addresses](https://krisconstable.com/generating-a-key-pair-with-nostr/). This is a great usecase for our notifyr decorator because calculating vanity addresses can be quite slow if we are looking for long words.

This simple module will let you look for an `npub` or a `hex` vanity address that starts with a particular pattern.

In [ ]:
#| export

from nostrfastr.nostr import PrivateKey
from nostrfastr.notifyr import notifyr
from nostr import bech32
import time
import secrets
import secp256k1
from numba import jit

Make the simple functions to guess public keys

In [ ]:
#| export

def guess_bytes():
    privkey_bytes = secrets.token_bytes(32) 
    sk = secp256k1.PrivateKey(privkey_bytes)
    pubkey_bytes = sk.pubkey.serialize()[1:]
    return privkey_bytes, pubkey_bytes

def guess_bech32(startswith=''):
    privkey_bytes, pubkey_bytes = guess_bytes()
    converted_bits = bech32.convertbits(pubkey_bytes, 8, 5)
    npub = bech32.bech32_encode("npub", converted_bits, bech32.Encoding.BECH32)
    if npub.startswith(startswith):
        return privkey_bytes.hex(), npub
    else:
        return None, None

def guess_hex(startswith=''):
    privkey_bytes, pubkey_bytes = guess_bytes()
    privkey_hex = privkey_bytes.hex()
    pubkey_hex = pubkey_bytes.hex()
    if pubkey_hex.startswith(startswith):
        return privkey_bytes.hex(), pubkey_bytes.hex()
    else:
        return None, None

In [ ]:
privkey_hex, npub = guess_bech32()
assert PrivateKey.from_hex(privkey_hex).public_key.bech32() == npub

In [ ]:
privkey_hex, pubkey_hex = guess_hex()
assert PrivateKey.from_hex(privkey_hex).public_key.hex() == pubkey_hex

Let's check the performance

In [ ]:
#| export

def time_guess(guesser):
    start = time.perf_counter()
    pub = guesser(startswith=' ')
    if pub is None: # this is just replicating what we will actually be doing
        pass
    end = time.perf_counter()
    interval = end - start
    return interval

def get_guess_rate(guesser, n_guesses=1e4):
    n_guesses = int(n_guesses)
    t = sum([time_guess(guesser) for _ in range(n_guesses)]) / n_guesses
    guesses_per_second = 1 / t
    return guesses_per_second

In [ ]:
get_guess_rate(guess_hex)

10828.338766288965

In [ ]:
get_guess_rate(guess_bech32)

4934.651268698682

In [ ]:
#| export

import math

In [ ]:
#| export

hex_chars = 'abcdef0123456789'
npub_chars = '023456789acdefghjklmnpqrstuvwxyz'


In [ ]:
#| export

def expected_guesses_by_char(p_char, num_char):
    return (p_char ** -num_char - 1)/ (1 - p_char)

def expected_time(num_char, options, time_per_guess):
    p = 1 / len(options)
    n_guess = expected_guesses_by_char(p, num_char)
    time_seconds = n_guess * time_per_guess
    return time_seconds

def expected_chars_by_time(p_char, num_guesses):
    n = - math.log(1 + (num_guesses * (1 - p_char))) / math.log(p_char)
    return n


In [ ]:
#| export

hex_chars = 'abcdef0123456789'
npub_chars = '023456789acdefghjklmnpqrstuvwxyz'

def average_char_by_time(options: str, time_per_guess: float):
    p = 1 / len(options)
    seconds_in_month = 60 * 60 * 24 * 30.5
    seconds_in_day = 60 * 60 * 24
    seconds_in_hour = 60 * 60
    seconds_in_minute = 60

    guesses_per_month = seconds_in_month / time_per_guess
    guesses_per_day = seconds_in_day / time_per_guess
    guesses_per_hour = seconds_in_hour / time_per_guess
    guesses_per_minute = seconds_in_minute / time_per_guess
    guesses_per_second = 1 / time_per_guess

    guesses = [guesses_per_second, guesses_per_minute,\
               guesses_per_hour, guesses_per_day, guesses_per_month]

    expected_chars = [expected_chars_by_time(p, g) for g in guesses]
    results = zip(['one second', 'one minute', 'one hour', 'one day', 'one month'],
                   expected_chars)
    for t, c in results:
        print(f'In {t} you can expect to get {c} characters on average')

def average_time_by_char(num_chars, time_per_guess):
    for n in range(20):
        n += 1
        t = expected_time(n, num_chars, time_per_guess)
        print(f'{n} characters: it might take {t} seconds')



In [ ]:
time_per_guess = 1 / get_guess_rate(guess_bech32)
average_char_by_time(npub_chars, time_per_guess)

In one second you can expect to get 2.4390896209265716 characters on average
In one minute you can expect to get 3.620407240081551 characters on average
In one hour you can expect to get 4.801784350762993 characters on average
In one day you can expect to get 5.718776834527163 characters on average
In one month you can expect to get 6.704924301350915 characters on average


In [ ]:
time_per_guess = 1 / get_guess_rate(guess_hex)
average_char_by_time(hex_chars, time_per_guess)

In one second you can expect to get 3.3569012406008594 characters on average
In one minute you can expect to get 4.833591699365952 characters on average
In one hour you can expect to get 6.310313811741457 characters on average
In one day you can expect to get 7.456554428206971 characters on average
In one month you can expect to get 8.689238762231213 characters on average


In [ ]:
average_time_by_char(hex_chars, time_per_guess)

1 characters: it might take 0.0013615036602364853 seconds
2 characters: it might take 0.02314556222402025 seconds
3 characters: it might take 0.3716904992445605 seconds
4 characters: it might take 5.948409491573204 seconds
5 characters: it might take 95.1759133688315 seconds
6 characters: it might take 1522.8159754049643 seconds
7 characters: it might take 24365.056967983088 seconds
8 characters: it might take 389840.9128492331 seconds
9 characters: it might take 6237454.606949233 seconds
10 characters: it might take 99799273.71254924 seconds
11 characters: it might take 1596788379.4021492 seconds
12 characters: it might take 25548614070.43575 seconds
13 characters: it might take 408777825126.9733 seconds
14 characters: it might take 6540445202031.575 seconds
15 characters: it might take 104647123232505.2 seconds
16 characters: it might take 1674353971720083.2 seconds
17 characters: it might take 2.6789663547521332e+16 seconds
18 characters: it might take 4.286346167603413e+17 seconds


In [ ]:
#| export

def expected_performance():
    print(
        '''This is a random guessing process - estimations are an average, but the actual
        time it takes to find a key could be significantly more or less than the estimate!
        Please keep that in mind when choosing an option.
        ''')
    print('hex:')
    time_per_guess_hex = 1 / get_guess_rate(guesser=guess_hex)
    average_char_by_time(hex_chars, time_per_guess_hex)
    print('\n')
    average_time_by_char(hex_chars, time_per_guess_hex)
    print('\n')

    print('npub:')
    time_per_guess_bech32 = 1 / get_guess_rate(guesser=guess_bech32)
    average_char_by_time(npub_chars, time_per_guess_bech32)
    print('\n')
    average_time_by_char(npub_chars, time_per_guess_bech32)
    print('\n')

    

In [ ]:
expected_performance()

This is a random guessing process - estimations are an average, but the actual
        time it takes to find a key could be significantly more or less than the estimate!
        Please keep that in mind when choosing an option.
        
hex:
In one second you can expect to get 3.2170015513860672 characters on average
In one minute you can expect to get 4.693676755553673 characters on average
In one hour you can expect to get 6.1703986136573485 characters on average
In one day you can expect to get 7.316639225992733 characters on average
In one month you can expect to get 8.549323559843291 characters on average


1 characters: it might take 0.0020067510750377553 seconds
2 characters: it might take 0.03411476827564184 seconds
3 characters: it might take 0.5478430434853072 seconds
4 characters: it might take 8.767495446839952 seconds
5 characters: it might take 140.28193390051428 seconds
6 characters: it might take 2244.5129491593034 seconds
7 characters: it might take 35912.20919329993 s

Lets make the function...

In [ ]:
#| export

def gen_vanity_pubkey(startswith: str, style='hex') -> PrivateKey:
    """randomly generate private keys until one matches the desire
    startswith for an npub or hex

    Parameters
    ----------
    startswith : str
        characters that the public key should start with. More chars
        means longer run time
    style : str, optional
        'npub' or 'hex' - npub is more commonly displayed on apps
        while hex is the true base private key with no encoding,
        by default 'hex'

    Returns
    -------
    PrivateKey
        returns a private key object
    """
    pubkey = None
    if style == 'npub':
        if not all(c in npub_chars for c in startswith):
            raise ValueError(f'character of selection not '
                              'in npub pattern ({npub_chars})')
        time_per_guess = 1 / get_guess_rate(guess_bech32)
        t = expected_time(len(startswith), npub_chars, time_per_guess)
        startswith = f'npub1{startswith}'
    else:
        if not all(c in hex_chars for c in startswith):
            raise ValueError(f'character of selection not in '
                              'hex pattern ({hex_chars})')
        time_per_guess = 1 / get_guess_rate(guess_hex)
        t = expected_time(len(startswith), hex_chars, time_per_guess)
    print(f'It might take {t} seconds to find a {style} pubkey that starts with '
          f'{startswith}. Note that this is a very rough estimate and due '
          'to the random nature of finding vanity keys it could take MUCH '
          'longer.')
    while pubkey is None:
        if style == 'npub':
            privkey_hex, pubkey = guess_bech32(startswith=startswith)
        else:
            privkey_hex, pubkey = guess_hex(startswith=startswith)
    return PrivateKey.from_hex(privkey_hex)

In [ ]:
from fastcore.test import test_fail

Make sure we don't allow characters that will never happen - test that these cases fail

In [ ]:
fail = lambda _: gen_vanity_pubkey(startswith='b', style='npub')
test_fail(fail)
fail = lambda _: gen_vanity_pubkey(startswith='g', style='hex')
test_fail(fail)


Generate a couple npubs!

In [ ]:
vanity_private_key_npub = gen_vanity_pubkey(startswith='23', style='npub')
vanity_private_key_hex = gen_vanity_pubkey(startswith='23', style='hex')

It might take 0.2441909272051882 seconds to find a npub pubkey that starts with npub123. Note that this is a very rough estimate and due to the random nature of finding vanity keys it could take MUCH longer.
It might take 0.02088132944642566 seconds to find a hex pubkey that starts with 23. Note that this is a very rough estimate and due to the random nature of finding vanity keys it could take MUCH longer.


And make sure it worked...

In [ ]:
assert vanity_private_key_npub.public_key.bech32().startswith('npub123')
assert vanity_private_key_hex.public_key.hex().startswith('23')

Now we can also make a version of this that notifies you.

In [ ]:
#| export

vanity_notifyr = notifyr(gen_vanity_pubkey)

Remember that if you want a vanity notifyr that will go to a different address than the one you find in `vanity_notifyr.notifyr_privkey_hex` you can create your own notifyr like so:
```python
new_vanity_notifyr = notifyr(gen_vanity_pubkey, recipient_address=your_address)
```

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()